In [22]:
import torch 
import torchvision
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from torchvision.models import resnet18, ResNet18_Weights

# 1. Basic autograd example 1

In [2]:
# create tensors
x = torch.tensor(1., requires_grad=True)
w = torch.tensor(2., requires_grad=True)
b = torch.tensor(3., requires_grad=True)

# build a computational graph
y = w * x + b

# compute gradients
y.backward()

# print out the gradients
print(x.grad)   # x.grad = 2
print(w.grad)   # w.grad = 1
print(b.grad)   # b.grad = 1

tensor(2.)
tensor(1.)
tensor(1.)


# 2. Basic autograd example 2

In [3]:
# Create tensors of shape (10, 3) and (10, 2).
x = torch.randn(10, 3)
y = torch.randn(10, 2)

# Build a fully connected layer.
linear = nn.Linear(3, 2)

# Build loss function and optimizer.
loss_func = nn.MSELoss()
optimizer = torch.optim.SGD(linear.parameters(), lr=0.01)

# Forward pass.
pred = linear(x)

# Compute loss.
loss = loss_func(pred, y)
print('loss:', loss.item())

# Backward pass.
loss.backward()

# Print out the gradients.
print('dL/dw:', linear.weight.grad)
print('dL/db:', linear.bias.grad)

# 1-step gradient descent.
optimizer.step()

# You can also perform gradient descent at the low level.
# linear.weight.data.sub_(0.01 * linear.weight.grad.data)
# linear.bias.data.sub_(0.01 * linear.bias.grad.data)

# Print out the loss after 1-step gradient descent.
pred = linear(x)
loss = loss_func(pred, y)
print('loss after 1 step update:', loss.item())   # it should decrease compared to the first-time loss

loss: 0.8465279340744019
dL/dw: tensor([[-0.4377,  0.3501,  0.2573],
        [ 0.4771,  0.2365,  0.0517]])
dL/db: tensor([ 0.2661, -0.1565])
loss after 1 step update: 0.8389414548873901


# 3. Loading data from numpy

In [10]:
# Create a numpy array.
x = np.array([[1,2,3], [4,5,6]])
print(x)

# Convert the numpy array to a torch tensor.
y = torch.from_numpy(x)
print(y)

# Convert the torch tensor to a numpy array.
z = y.numpy()
print(z)

[[1 2 3]
 [4 5 6]]
tensor([[1, 2, 3],
        [4, 5, 6]])
[[1 2 3]
 [4 5 6]]


# 4. Input pipeline

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [14]:
# Download and construct CIFAR-10 dataset.
data_dir = '/content/drive/My Drive/PyTorch/Github_Series/01-basics/'
train_dataset = torchvision.datasets.CIFAR10(root=data_dir,
                                             train=True, 
                                             transform=transforms.ToTensor(),
                                             download=True)

# Fetch one data pair (read data from disk).
image, label = train_dataset[0]
print(image.size())
print(label)

Files already downloaded and verified
torch.Size([3, 32, 32])
6


In [13]:
# Data loader (this provides queues and threads in a very simple way).
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# When iteration starts, queue and thread start to load data from files.
data_iter = iter(train_loader)

# Mini-batch images and labels.
images, labels = data_iter.next()

# Actual usage of the data loader is as below.
for images, labels in train_loader:
    # Training code should be written here.
    pass

# 5. Input pipeline for custom dataset

In [20]:
# You should build your custom dataset as below.
class CustomDataset(torch.utils.data.Dataset):
  def __init__(self):
    # TODO
    # 1. Initialize file paths or a list of file names.
    pass
  def __getitem__(self, index):
    # TODO
    # 1. Read one data from file (e.g. using numpy.fromfile, PIL.Image.open)
    # 2. Preprocess the data (e.g. torchvision.Transform)
    # 3. Return a data pair (e.g. image and label)
    pass
  def __len__(self):
    # TODO
    # 1. Return the toal size of your dataset
    return 1

# You can then use the prebuilt data loader. 
custom_dataset = CustomDataset()
train_loader = torch.utils.data.DataLoader(dataset=custom_dataset,
                                           batch_size=64, 
                                           shuffle=True)

# 6. Pretrained model

In [29]:
# Download and load the pretrained ResNet-18.
resnet18 = torchvision.models.resnet18(weights=ResNet18_Weights.DEFAULT)    # PyTorch v1.5+

# If you want to finetune only the top layer of the model, set as below.
for param in resnet18.parameters():
  param.requires_grad = False

# Display all model layer weights
# for name, para in resnet18.named_parameters():
#     print('{}: {}'.format(name, para.shape))

# Replace the top layer for finetuning.
resnet18.fc = nn.Linear(resnet18.fc.in_features, 100)  # 100 is an example.

# Forward pass.
images = torch.randn(64, 3, 224, 224)     
output = resnet18(images)
print(output.size())    # (64, 100)

torch.Size([64, 100])


# 7. Save and load the model  

In [28]:
# Save and load the entire model.
torch.save(resnet18, data_dir + 'models/resnet18.ckpt')
resnet18 = torch.load(data_dir + 'models/resnet18.ckpt')

# Save and load only the model parameters (recommended).
torch.save(resnet18.state_dict(), data_dir + 'models/params.ckpt')
resnet18.load_state_dict(torch.load(data_dir + 'models/params.ckpt'))

<All keys matched successfully>